In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 
from peft import PeftConfig, PeftModel
from tqdm import tqdm
from statistics import mean

/u/prasanns/miniconda3/envs/fixenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
bonds = pd.read_json("nmathgsdpo.json", lines=True, orient='records')

In [ ]:
adapt_path = "../checkpoints/math/mbestofnsft_sft_bonsft/final_checkpoint/"
base = "../outputs/models/math/randbigsft/"
# load in adapter / model
peft_config = PeftConfig.from_pretrained(adapt_path)
model = AutoModelForCausalLM.from_pretrained(
    base, load_in_8bit=True, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(base)

# Load the Lora model
model = PeftModel.from_pretrained(model, adapt_path)
model.eval()

In [9]:
# load in data to get likelihoods on
bonds = pd.read_json("nmath.json", lines=True, orient='records')

In [13]:
def calculate_nll(input_string):
    # Tokenize the input string
    inputs = tokenizer(input_string, return_tensors='pt')
    
    # Get input_ids and move to the appropriate device
    input_ids = inputs['input_ids']
    device = model.device
    input_ids = input_ids.to(device)
    
    # Get the logits from the model
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    
    # Calculate the loss
    loss = outputs.loss
    
    # Convert the loss to NLL
    nll = loss.item()
    
    return nll

In [19]:
glikes = []
for i, r in tqdm(bonds.iterrows(), total=len(bonds)):
    tmplikes = [calculate_nll(r['ags'][ind]) for ind in range(len(r['ags']))]
    glikes.append(tmplikes)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 739/739 [05:13<00:00,  2.36it/s]


In [13]:
bonds['bonsft_likes'] = glikes

In [14]:
# bonds.to_json("nmath.json", lines=True, orient='records')

In [28]:
glikes = bonds['bonsft_likes']

In [29]:
mean([mean(m) for m in glikes])

1.044623343512388

In [30]:
mean([m[0] for m in glikes])

1.040043026894452